In [1]:
import sys
import json
import math
import pyvista as pv
import numpy as np
import matplotlib.pyplot as plt
import pprint
pp = pprint.PrettyPrinter(indent=2)

filename = "/Users/ken/Downloads/montreal_final.json"

## Read data

In [2]:
with open(filename) as file:
    cm = json.load(file)
    
len(cm["CityObjects"])

28018

## Totals

In [3]:
totals = {}
for co_id, co in cm["CityObjects"].items():
    if co["type"] in totals:
        totals[co["type"]] += 1
    else:
        totals[co["type"]] = 1
        
for co_type, co_total in totals.items():
    print(str(co_type) + ": " + str(co_total))

Road: 4387
Building: 19430
LandUse: 4120
PlantCover: 80
WaterBody: 1


## Load roads and buildings into PyVista

In [4]:
roads = {}
buildings = {}

for co_id in list(cm["CityObjects"]):
    co = cm["CityObjects"][co_id]
    
    if co["type"] == "Road":
        roads[co_id] = {}
        for geom in co["geometry"]:
            if geom["type"] == "MultiLineString":
                geom_line = geom
            if geom["type"] == "MultiSurface":
                geom_surface = geom
                
        lines = []
        lines_vertices = []
        num_lines = 0
        for segment in geom_line["boundaries"]:
            lines.extend([2, 2*num_lines, 2*num_lines+1])
            lines_vertices.extend([cm["vertices"][segment[0]], cm["vertices"][segment[1]]])
            num_lines += 1
        if len(lines) > 0:
            mesh_lines = pv.PolyData(lines_vertices, lines=lines, n_lines=len(geom_line["boundaries"]))
            roads[co_id]["line"] = mesh_lines
        
        surface = []
        surface_vertices = []
        num_triangles = 0
        for triangle in geom_surface["boundaries"]:
            surface.extend([3, 3*num_triangles, 3*num_triangles+1, 3*num_triangles+2])
            surface_vertices.extend([cm["vertices"][triangle[0][0]], cm["vertices"][triangle[0][1]], cm["vertices"][triangle[0][2]]])
            num_triangles += 1
        if len(surface) > 0:
            mesh_surface = pv.PolyData(surface_vertices, surface, len(geom_surface["boundaries"]))
            roads[co_id]["surface"] = mesh_surface
            
    if co["type"] == "Building":
        buildings[co_id] = {}
        for geom in co["geometry"]:
            if geom["type"] == "Solid":
                geom_solid = geom
        
        surface = []
        surface_vertices = []
        num_triangles = 0
        for triangle in geom_solid["boundaries"][0]:
            surface.extend([3, 3*num_triangles, 3*num_triangles+1, 3*num_triangles+2])
            surface_vertices.extend([cm["vertices"][triangle[0][0]], cm["vertices"][triangle[0][1]], cm["vertices"][triangle[0][2]]])
            num_triangles += 1
        if len(surface) > 0:
            mesh_surface = pv.PolyData(surface_vertices, surface, len(geom_surface["boundaries"]))
            buildings[co_id]["surface"] = mesh_surface
        

## Remove roads and buildings without required geometries

In [5]:
for road_id in list(roads):
    if "surface" not in roads[road_id] or "line" not in roads[road_id]:
        roads.pop(road_id)
        
for building_id in list(buildings):
    if "surface" not in buildings[building_id]:
        buildings.pop(building_id)
        
print(str(len(roads)) + " roads")
print(str(len(buildings)) + " buildings")

1968 roads
19430 buildings


## Compute widths

In [6]:
def bbox_distance(pd1, pd2):
    xmin1 = pd1.bounds[0]
    xmax1 = pd1.bounds[1]
    ymin1 = pd1.bounds[2]
    ymax1 = pd1.bounds[3]
    zmin1 = pd1.bounds[4]
    zmax1 = pd1.bounds[5]
    xmin2 = pd2.bounds[0]
    xmax2 = pd2.bounds[1]
    ymin2 = pd2.bounds[2]
    ymax2 = pd2.bounds[3]
    zmin2 = pd2.bounds[4]
    zmax2 = pd2.bounds[5]
    if xmin1 > xmax2:
        xdist = xmin1-xmax2
    elif xmax1 < xmin2:
        xdist = xmin2-xmax1
    else:
        xdist = 0.0
    if ymin1 > ymax2:
        ydist = ymin1-ymax2
    elif ymax1 < ymin2:
        ydist = ymin2-ymax1
    else:
        ydist = 0.0
    if zmin1 > zmax2:
        zdist = zmin1-zmax2
    elif zmax1 < zmin2:
        zdist = zmin2-zmax1
    else:
        zdist = 0.0
    return math.sqrt(xdist*xdist + ydist*ydist + zdist*zdist)

In [ ]:
search_radius = 20.0 # for each road, use buildings roughly this distance away (bounding boxes)
view_height = 5.0 # how high the viewpoint is
slice_interval = 5.0 # distance between slices
angles_to_test = 36 # how many angles to test
output_path = "/Users/ken/Downloads/road_stats.csv"

viewpoint = np.array([0, 0, view_height])
angle_interval = int(360/angles_to_test)
output_file = open(output_path, "w")
output_file.write("road_id,nearby_buildings,minimum_distance_mean,minimum_distance_min,maximum_distance_mean,maximum_distance_max,maximum_obscured_angle_mean,maximum_obscured_angle_max,sky_visibility_mean,sky_visibility_min,sky_visibility_max\n")
for road_id in list(roads): # limited for testing
    print(road_id)
    
    # For each road, find nearby buildings
    nearby_buildings = []
    for building_id in list(buildings):
        if bbox_distance(roads[road_id]["surface"], buildings[building_id]["surface"]) < search_radius:
            nearby_buildings.append(building_id)
    if len(nearby_buildings) == 0:
        continue
    roads[road_id]["nearby_buildings"] = nearby_buildings # stored as IDs
    
    # Create one mesh with nearby buildings
    vertices = []
    faces = []
    num_triangles = 0
    for building_id in roads[road_id]["nearby_buildings"]:
        for i in range(int(len(buildings[building_id]["surface"].faces)/4)):
            vertices.extend([buildings[building_id]["surface"].points[buildings[building_id]["surface"].faces[4*i+1]],
                             buildings[building_id]["surface"].points[buildings[building_id]["surface"].faces[4*i+2]],
                             buildings[building_id]["surface"].points[buildings[building_id]["surface"].faces[4*i+3]]])
            faces.extend([3, 3*num_triangles, 3*num_triangles+1, 3*num_triangles+2])
            num_triangles += 1
    nearby_buildings = pv.PolyData(vertices, faces, num_triangles)
    
    # Compute slices
    points_along_line = []
    line_slices = []
    line_slices_2d = []
    for i in range(int(len(roads[road_id]["line"].lines)/3)):
        line_start = roads[road_id]["line"].points[roads[road_id]["line"].lines[3*i+1]]
        line_end = roads[road_id]["line"].points[roads[road_id]["line"].lines[3*i+2]]
        line_vector = line_end-line_start # used to cut orthogonally along road line
        line_vector[2] = 0.0 # make sure the slices are vertical
        norm = np.linalg.norm(line_vector)
        line_vector /= norm
        start = (norm-slice_interval*math.floor(norm/slice_interval))/2.0
        for j in np.arange(start, norm, slice_interval):
            points_along_line.append(line_vector*j+line_start)
            line_slices.append(nearby_buildings.slice(line_vector, points_along_line[-1]))
            centred_points = line_slices[-1].points-points_along_line[-1]
            points_2d = []
            for p in centred_points:
                magnitude = math.sqrt(p[0]*p[0]+p[1]*p[1])
                orientation = math.atan2(p[1], p[0])
                if orientation > -0.5*math.pi and orientation < 0.5*math.pi:
                    points_2d.append([magnitude, 0.0, p[2]])
                else:
                    points_2d.append([-magnitude, 0.0, p[2]])
            line_slices_2d.append(pv.PolyData(points_2d, lines=line_slices[-1].lines, n_lines=int(len(line_slices[-1].lines)/3)))
    roads[road_id]["slice_points"] = points_along_line
    roads[road_id]["slices"] = line_slices
    roads[road_id]["slices_2d"] = line_slices_2d
    
    # Compute minimum distances for each angle
    roads[road_id]["viewpoint_distances"] = []
    for road_slice in roads[road_id]["slices_2d"]:
        roads[road_id]["viewpoint_distances"].append({})
        for angle in range(0, 360, angle_interval):
            roads[road_id]["viewpoint_distances"][-1][int(angle)] = math.inf
        
        for i in range(int(len(road_slice.lines)/3)):
#             print("Viewpoint:" + str(viewpoint))
            line_start = road_slice.points[road_slice.lines[3*i+1]]-viewpoint
            line_end = road_slice.points[road_slice.lines[3*i+2]]-viewpoint
            # print("Line: " + str(line_start) + " to " + str(line_end))
            line_start_angle = 180.0*math.atan2(line_start[2], line_start[0])/math.pi
            if line_start_angle < 0.0:
                line_start_angle += 360.0
            line_end_angle = 180.0*math.atan2(line_end[2], line_end[0])/math.pi
            if line_end_angle < 0.0:
                line_end_angle += 360.0
            angles_diff = line_end_angle-line_start_angle
            if angles_diff > -180.0 and angles_diff < 0.0: # always do ccw
                line_start, line_end = line_end, line_start
                line_start_angle, line_end_angle = line_end_angle, line_start_angle
                angles_diff = line_end_angle-line_start_angle
            # print("\tangles: " + str(line_start_angle) + " to " + str(line_end_angle) + " diff: " + str(angles_diff))
            iteration_start = math.ceil(line_start_angle/angle_interval)*angle_interval
            iteration_end = math.floor(line_end_angle/angle_interval)*angle_interval
            # print("\titeration: " + str(iteration_start) + " to " + str(iteration_end))
            if angles_diff < 0.0:
                iteration_range = [*range(iteration_start, 360, angle_interval)]
                iteration_range.extend(range(0, iteration_end+angle_interval, angle_interval))
            else: # cases where we have 0 degrees in the middle
                iteration_range = range(iteration_start, iteration_end+angle_interval, angle_interval)
            for j in iteration_range:
                v1 = np.array([-line_start[0], -line_start[2]])
                v2 = np.array([line_end[0]-line_start[0], line_end[2]-line_start[2]])
                v3 = np.array([-math.sin(j*math.pi/180.0), math.cos(j*math.pi/180.0)])
                t1 = np.cross(v2, v1) / np.dot(v2, v3)
                t2 = np.dot(v1, v3) / np.dot(v2, v3)
                if t1 >= 0.0 and t2 >= 0.0 and t2 <= 1.0:
                    # print("\t\t" + str(j) + ": " + str(t1))
                    if t1 < roads[road_id]["viewpoint_distances"][-1][j]:
                        roads[road_id]["viewpoint_distances"][-1][j] = t1
                   
    # Compute stats
    maximum_obscured_angle = [] # steepest angle where we can see a building
    minimum_distance = [] # closest distance to visible building
    maximum_distance = [] # farthest distance to visible building (within search threshold)
    sky_visibility = [] # how many angles hit the sky
    for vp in roads[road_id]["viewpoint_distances"]:
        minimum_distance.append(math.inf)
        maximum_distance.append(0.0)
        maximum_obscured_angle.append(0.0)
        visible_visibility_angles = 0
        total_visibility_angles = 0
        first_angle = -1
        for angle, distance in vp.items():
            if distance < minimum_distance[-1]:
                minimum_distance[-1] = distance
            if distance > maximum_distance[-1] and distance < math.inf:
                maximum_distance[-1] = distance
            if angle > 0 and angle < 180:
                if distance < math.inf and abs(90-angle) < 90-maximum_obscured_angle[-1]:
                    if angle > 90:
                        maximum_obscured_angle[-1] = 180-angle
                    else:
                        maximum_obscured_angle[-1] = angle
            if angle >= 0 and angle <= 180:
                if first_angle == -1:
                    first_angle = angle
                last_angle = angle
                total_visibility_angles += 1
                if distance == math.inf:
                    visible_visibility_angles += 1
        if vp[first_angle] < math.inf and vp[last_angle] < math.inf:
            sky_visibility.append(float(visible_visibility_angles)/total_visibility_angles)
                    
    # Aggregate stats
    roads[road_id]["stats"] = {}
    roads[road_id]["stats"]["nearby_buildings"] = len(roads[road_id]["nearby_buildings"])
    roads[road_id]["stats"]["minimum_distance"] = {}
    roads[road_id]["stats"]["minimum_distance"]["mean"] = np.mean(minimum_distance)
    roads[road_id]["stats"]["minimum_distance"]["min"] = np.amin(minimum_distance)
    roads[road_id]["stats"]["maximum_distance"] = {}
    roads[road_id]["stats"]["maximum_distance"]["mean"] = np.mean(maximum_distance)
    roads[road_id]["stats"]["maximum_distance"]["max"] = np.amax(maximum_distance)
    roads[road_id]["stats"]["maximum_obscured_angle"] = {}
    roads[road_id]["stats"]["maximum_obscured_angle"]["mean"] = np.mean(maximum_obscured_angle)
    roads[road_id]["stats"]["maximum_obscured_angle"]["max"] = np.amax(maximum_obscured_angle)
    roads[road_id]["stats"]["sky_visibility"] = {}
    if len(sky_visibility) > 0:
        roads[road_id]["stats"]["sky_visibility"]["mean"] = np.mean(sky_visibility)
        roads[road_id]["stats"]["sky_visibility"]["min"] = np.amin(sky_visibility)
        roads[road_id]["stats"]["sky_visibility"]["max"] = np.amax(sky_visibility)
    else:
        roads[road_id]["stats"]["sky_visibility"]["mean"] = np.NaN
        roads[road_id]["stats"]["sky_visibility"]["min"] = np.NaN
        roads[road_id]["stats"]["sky_visibility"]["max"] = np.NaN
    
    # Write stats to file
    output_file.write(road_id)
    output_file.write(",")
    output_file.write(str(roads[road_id]["stats"]["minimum_distance"]["mean"]))
    output_file.write(",")
    output_file.write(str(roads[road_id]["stats"]["minimum_distance"]["min"]))
    output_file.write(",")
    output_file.write(str(roads[road_id]["stats"]["maximum_distance"]["mean"]))
    output_file.write(",")
    output_file.write(str(roads[road_id]["stats"]["maximum_distance"]["max"]))
    output_file.write(",")
    output_file.write(str(roads[road_id]["stats"]["maximum_obscured_angle"]["mean"]))
    output_file.write(",")
    output_file.write(str(roads[road_id]["stats"]["maximum_obscured_angle"]["max"]))
    output_file.write(",")
    output_file.write(str(roads[road_id]["stats"]["sky_visibility"]["mean"]))
    output_file.write(",")
    output_file.write(str(roads[road_id]["stats"]["sky_visibility"]["min"]))
    output_file.write(",")
    output_file.write(str(roads[road_id]["stats"]["sky_visibility"]["max"]))
    output_file.write("\n")
    
    # Delete temporary data
    roads[road_id].pop("nearby_buildings")
    roads[road_id].pop("slice_points")
    roads[road_id].pop("slices")
    roads[road_id].pop("slices_2d")
    roads[road_id].pop("viewpoint_distances")
      
output_file.close()
# pp.pprint(roads[list(roads)[50]])

100055890
100055897
100056043
100056045
100056047
100056049
100056050
100056055
100056061
100056062
100056063
100056068
100056073
100056075
100056076
100056078
100056079
100056085
100056090
100056092
100056098
100056102
100056109
100056119
100056121
100056123
100056124
100056130
100056132
100056137
100056139
100056151
100056152
100056161
100056164
100056166
100056170
100056173
100056174
100056179
100056180
100056185
100056186
100056189
100056192
100056194
100056195
100056198
100056199
100056201
100056204
100056206
100056214
100056220
100056222
100056224
100056225
100056227
100056228
100056230
100056232
100056234
100056239
100056240
100056242
100056243
100056244
100056246
100056249
100056253
100056255
100056256
100056257
100056259
100056263
100056265
100056267
100056268
100056271
100056273
100056275
100056278
100056279
100056282
100056284
100056285
100056288
100056289
100056294
100056295
100056296
100056297
100056299
100056303
100056304
100056307
100056309
100056310
100056317
100056321


## Plots: nearby buildings

In [ ]:
p = pv.Plotter()

for road_id in list(roads)[2:3]: # nice example road
    p.add_mesh(roads[road_id]["surface"], color="black")
    p.add_mesh(roads[road_id]["line"], color="yellow", line_width=2)
    if "nearby_buildings" in roads[road_id]:
        for building_id in roads[road_id]["nearby_buildings"]:
            p.add_mesh(buildings[building_id]["surface"], color="red", show_edges=True)

p.show()

## Plots: slices

In [ ]:
p = pv.Plotter()

for road_id in list(roads)[2:3]: # nice example road
    p.add_mesh(roads[road_id]["surface"], color="black")
    p.add_mesh(roads[road_id]["line"], color="yellow", line_width=2)
    if "nearby_buildings" in roads[road_id]:
        for building_id in roads[road_id]["nearby_buildings"]:
            p.add_mesh(buildings[building_id]["surface"], color="white", opacity=0.5)
    if "slices" in roads[road_id]:
        for s in roads[road_id]["slices"]:
            p.add_mesh(s, color="blue", line_width=2)
    points = pv.PolyData(roads[road_id]["slice_points"])
    p.add_points(points, color="red")
p.show()

## Plots: slices in 2D

In [ ]:
figure = 1

for road_id in list(roads)[2:3]: # nice example road
    for road_slice in roads[road_id]["slices_2d"]:
        plt.figure(figure)
        figure += 1
        for i in range(int(len(road_slice.lines)/3)):
            line_start = road_slice.points[road_slice.lines[3*i+1]]
            line_end = road_slice.points[road_slice.lines[3*i+2]]
            plt.plot([line_start[0], line_end[0]], [line_start[2], line_end[2]], 
                     'b-', lw=2)
        plt.plot(0.0, view_height, 'ro')
        plt.axis('scaled')
        plt.show()

## Plots: slices in 2D (polar coordinates)

In [ ]:
for road_id in list(roads)[2:3]: # nice example road
    for road_slice in roads[road_id]["slices_2d"]:
        plt.figure(figure)
        figure += 1
        viewpoint = np.array([0, 0, view_height])
        ax = plt.subplot(111, projection='polar')
        for i in range(int(len(road_slice.lines)/3)):
            line_start = road_slice.points[road_slice.lines[3*i+1]]-viewpoint
            line_end = road_slice.points[road_slice.lines[3*i+2]]-viewpoint
            
            line_start_polar = [math.atan2(line_start[2], line_start[0]),
                                math.sqrt(line_start[0]*line_start[0]+line_start[2]*line_start[2])]
            line_end_polar = [math.atan2(line_end[2], line_end[0]),
                                math.sqrt(line_end[0]*line_end[0]+line_end[2]*line_end[2])]
            plt.plot([line_start_polar[0], line_end_polar[0]], [line_start_polar[1], line_end_polar[1]], 
                     'b-', lw=2)
        plt.plot(0.0, 0.0, 'ro')
        plt.show()


# Plots: minimum distances for angles

In [ ]:
for road_id in list(roads)[2:3]: # nice example road
    for road_slice in roads[road_id]["viewpoint_distances"]:
        plt.figure(figure)
        figure += 1
        for angle in road_slice:
            if road_slice[angle] < math.inf:
                # print(str(angle) + ": " + str(road_slice[angle]))
                plt.polar(math.pi*angle/180.0, road_slice[angle]/180.0, 'bo')
        plt.show()